In [1]:
#  Import a bunch of stuff
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['image.cmap'] = 'hot'
import IPython.display as ipd
import tensorflow as tf
import mir_eval
import importlib
from datetime import datetime
import soundfile as sf

import audio_functions as af
import audio_models
import dataset
import model_functions as mf

C:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
importlib.reload(audio_models)

In [8]:
#  Set variables
sample_rate=16384
n_fft=1024
fft_hop=256
patch_window=256
patch_hop=128
n_parallel_readers=16
normalise=True
batch_size = 1
shuffle=False
n_shuffle = 1
mag_phase = True
learning_rate = 0.0002
data_type = 'mag_phase'

#checkpoint = '174/174-8'
#model_base_dir = '/home/enterprise.internal.city.ac.uk/acvn728/checkpoints'

directory_a = '/home/enterprise.internal.city.ac.uk/acvn728/miniCHiME/Mixed'
directory_b = '/home/enterprise.internal.city.ac.uk/acvn728/miniCHiME/Voice'
directory_c = '/home/enterprise.internal.city.ac.uk/acvn728/miniCHiME/BAckground'

#directory_a = 'C:/Users/Toby/Speech_Data/BG_test/tr05_bus_simu'
#directory_b = 'C:/Users/Toby/Speech_Data/BG_test/tr05_org'
#directory_c = 'C:/Users/Toby/Speech_Data/BG_test/tr05_bus_bg'

#  Create the pipeline
tf.reset_default_graph()
data = dataset.zip_files(directory_a, directory_b, directory_c)
data = dataset.get_paired_dataset(data,
                                  sample_rate,
                                  n_fft,
                                  fft_hop,
                                  patch_window,
                                  patch_hop,
                                  n_parallel_readers,
                                  batch_size,
                                  n_shuffle,
                                  normalise)

#  Create the iterator
pipe = data.make_initializable_iterator()
mixed_spec, voice_spec, background_spec, mixed_audio, voice_audio, background_audio = pipe.get_next()

#  Create variable placeholders
is_training = tf.placeholder(shape=(), dtype=bool)
mixed_phase = tf.expand_dims(mixed_spec[:, :, :-1, 3], 3)
if data_type == 'mag':
    mixed_input = tf.expand_dims(mixed_spec[:, :, :-1, 2], 3)
    voice_input = tf.expand_dims(voice_spec[:, :, :-1, 2], 3)
elif data_type in ['mag_phase', 'mag_phase_diff']:
    mixed_input = mixed_spec[:, :, :-1, 2:4]
    voice_input = voice_spec[:, :, :-1, 2:4]
elif data_type == 'real_imag':
    mixed_input = mixed_spec[:, :, :-1, 0:2]
    voice_input = voice_spec[:, :, :-1, 0:2]
elif data_type == 'mag_real_imag':
    mixed_input = tf.concat([tf.expand_dims(mixed_spec[:, :, :-1, 2], 3), mixed_spec[:, :, :-1, 0:2]], 3)
    voice_input = tf.concat([tf.expand_dims(voice_spec[:, :, :-1, 2], 3), voice_spec[:, :, :-1, 0:2]], 3)
elif data_type == 'mag_phase_real_imag':
    mixed_input = mixed_spec[:, :, :-1, :]
    voice_input = voice_spec[:, :, :-1, :]

data

<PrefetchDataset shapes: ((?, 256, 513, 4), (?, 256, 513, 4), (?, 256, 513, 4), (?, 65280, 1), (?, 65280, 1), (?, 65280, 1)), types: (tf.float32, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32)>

In [9]:
# Build U-Net model
print('Creating model')
model = audio_models.MagnitudeModel(mixed_input, voice_input, mixed_phase, mixed_audio, 
                                    voice_audio, background_audio, '3Dunet', is_training, learning_rate, 
                                    data_type, name='Magnitude_Model')
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
tf_config.gpu_options.visible_device_list = str(1)
sess = tf.Session(config=tf_config)
sess.run(tf.global_variables_initializer())

Creating model


In [3]:
enc = audio_models.UNet3DEncoder(mixed_input, is_training, reuse=False)
dec = audio_models.UNet3DDecoder(enc.output, enc, data_type, is_training, reuse=False)

In [4]:
print(enc.input_tensor.shape,'\n',
     enc.l1.shape,'\n',
     enc.l2.shape,'\n',
     enc.l3.shape,'\n',
     enc.l4.shape,'\n',
     enc.l5.shape,'\n',
     enc.output.shape)

(?, 256, 512, 2, 1) 
 (?, 128, 256, 2, 8) 
 (?, 64, 128, 2, 16) 
 (?, 32, 64, 2, 32) 
 (?, 16, 32, 2, 64) 
 (?, 8, 16, 2, 128) 
 (?, 4, 8, 2, 256)


In [5]:
print(dec.input_tensor.shape,'\n',
     dec.l1.shape,'\n',
     dec.l2.shape,'\n',
     dec.l3.shape,'\n',
     dec.l4.shape,'\n',
     dec.l5.shape,'\n',
     dec.output.shape)

(?, 4, 8, 2, 256) 
 (?, 8, 16, 2, 128) 
 (?, 16, 32, 2, 64) 
 (?, 32, 64, 2, 32) 
 (?, 64, 128, 2, 16) 
 (?, 128, 256, 2, 8) 
 (?, 256, 512, 2)


In [10]:
sess.run(pipe.initializer)
print('start: {}'.format(datetime.now()))
epochs = 10
epoch = 0
epoch_cost = []
epoch_m_loss = []
epoch_p_loss = []
i = 0
while epoch < epochs:
    try:
        #_, cost = sess.run([model.train_op, model.cost], {model.is_training:True})
        _, cost, m_loss, p_loss = sess.run([model.train_op, model.cost, model.mag_loss, model.phase_loss], {model.is_training:True})
        #print('Iteration {i}: {dt}\nCost: {c}'.format(i=i, dt=datetime.datetime.now(), c=cost))
        epoch_cost.append(cost)
        epoch_m_loss.append(m_loss)
        epoch_p_loss.append(p_loss)
        i += 1
    except tf.errors.OutOfRangeError:
        epoch_mean_cost = sum(epoch_cost)/len(epoch_cost)
        epoch_mean_m_loss = sum(epoch_m_loss)/len(epoch_m_loss)
        epoch_mean_p_loss = sum(epoch_p_loss)/len(epoch_p_loss)
        #print('Epoch {e} finished. Mean cost = {emc}'.format(e=epoch, emc=epoch_mean_cost))
        print('\tMean magnitude loss = {emml}\n\tMean phase loss = {empl}'.format(emml=epoch_mean_m_loss,
                                                                               empl=epoch_mean_p_loss))
        epoch += 1
        sess.run(pipe.initializer)
print('finish: {}'.format(datetime.now()))

start: 2018-11-08 11:53:43.073831


ZeroDivisionError: division by zero